# KServe Model Onboarding

## Welcome to the Self-Healing Platform ML Integration!

This notebook guides you through using KServe models via the Coordination Engine.

### What You'll Learn

1. **Connect** to the Coordination Engine
2. **Discover** available KServe models
3. **Call** models for predictions
4. **Add** custom models (no code changes needed!)

### Architecture Overview

```
┌─────────────┐     ┌──────────────────────┐     ┌─────────────────────┐
│  Notebook   │────▶│ Coordination Engine  │────▶│ KServe Inference    │
│  (You!)     │     │ /api/v1/detect       │     │ Services            │
└─────────────┘     └──────────────────────┘     └─────────────────────┘
```

**Benefits:**
- ✅ Central orchestration for all ML models
- ✅ Config-driven model registration
- ✅ No code changes to add new models
- ✅ GitOps-native workflow

---

**Reference:** ADR-039, ADR-040, [GitHub Issue #18](https://github.com/tosin2013/openshift-coordination-engine/issues/18)


## Step 1: Install Dependencies

First, ensure you have the required packages:


In [ ]:
# Install required packages (if not already installed)
import sys
!{sys.executable} -m pip install requests pandas numpy --quiet


## Step 2: Import the Coordination Engine Client


In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd

# Add the utils directory to the path
sys.path.insert(0, '../utils')

# Import the coordination engine client
from coordination_engine_client import (
    CoordinationEngineClient,
    DetectResponse,
    ModelHealth,
    get_client
)

print("✅ Coordination Engine Client imported successfully!")


## Step 3: Connect to the Coordination Engine

The client automatically discovers the coordination engine URL from:
1. `COORDINATION_ENGINE_URL` environment variable
2. Default: `http://coordination-engine:8080`


In [ ]:
# Create a client instance
client = get_client()
print(f"📡 Connected to: {client.base_url}")

# Verify connection with health check
try:
    health = client.health_check()
    print(f"✅ Coordination Engine is healthy!")
    print(f"   Version: {health.get('version', 'unknown')}")
    print(f"   Status: {health.get('status', 'unknown')}")
except Exception as e:
    print(f"⚠️  Could not connect: {e}")
    print("   Make sure the coordination engine is running.")


## Step 4: Discover Available Models

Let's see what KServe models are registered with the coordination engine:


In [ ]:
# List all registered models
try:
    models = client.list_models()
    model_count = client.get_model_count()
    
    print(f"📊 Found {model_count} registered model(s):")
    print("─" * 40)
    
    for i, model in enumerate(models, 1):
        print(f"   {i}. {model}")
    
    if not models:
        print("   No models registered yet.")
        print("   See 'Adding Custom Models' section below.")
except Exception as e:
    print(f"❌ Error listing models: {e}")


## Step 5: Make Predictions

Now let's call a model for predictions! The coordination engine supports the KServe v1 protocol.

### Prediction Format

**Request:**
```json
{
  "model": "anomaly-detector",
  "instances": [[0.5, 1.2, 0.8], [0.3, 0.9, 1.1]]
}
```

**Response:**
```json
{
  "predictions": [-1, 1],
  "model_name": "anomaly-detector",
  "model_version": "v1"
}
```

- `predictions`: `-1` = anomaly detected, `1` = normal


In [ ]:
# Example: Call the anomaly detector model
MODEL_NAME = "anomaly-detector"  # Change this to your model

# Sample data - replace with your actual feature vectors
sample_instances = [
    [0.5, 1.2, 0.8],   # Example: normal behavior
    [0.3, 0.9, 1.1],   # Example: normal behavior
    [2.5, 3.0, 4.0],   # Example: anomalous behavior
    [0.4, 1.0, 0.9],   # Example: normal behavior
    [5.0, 6.0, 7.0],   # Example: anomalous behavior
]

try:
    # Make prediction
    result = client.detect_anomaly(
        model_name=MODEL_NAME,
        instances=sample_instances
    )
    
    print(f"🔮 Predictions from '{result.model_name}':")
    print("─" * 50)
    
    for i, (instance, pred) in enumerate(zip(sample_instances, result.predictions)):
        status = "🔴 ANOMALY" if pred == -1 else "🟢 Normal"
        print(f"   Instance {i+1}: {instance} → {status}")
    
    print("─" * 50)
    print(f"📊 Summary:")
    print(f"   Total instances: {len(result.predictions)}")
    print(f"   Anomalies found: {result.anomaly_count()}")
    print(f"   Anomaly rate: {result.anomaly_rate():.1%}")
    
except Exception as e:
    print(f"❌ Prediction failed: {e}")
    print("\n💡 Tips:")
    print(f"   - Check if model '{MODEL_NAME}' is registered")
    print("   - Verify the model service is healthy")
    print("   - Check your feature vector format")


## Step 6: Adding Custom Models (No Code Changes!)

The coordination engine supports dynamic model discovery. You can add custom models by simply updating `values-hub.yaml`.

### How It Works

1. **Deploy your KServe InferenceService**
2. **Update values-hub.yaml** to register the model
3. **Sync with ArgoCD** - model is automatically available!

### Example: Adding a Disk Failure Predictor

**1. Deploy the InferenceService:**

```yaml
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: disk-failure-predictor
  namespace: self-healing-platform
spec:
  predictor:
    model:
      modelFormat:
        name: sklearn
      storageUri: "pvc://model-storage-pvc/disk-failure-predictor"
```

**2. Register in values-hub.yaml:**

```yaml
coordinationEngine:
  kserve:
    enabled: true
    namespace: self-healing-platform
    services:
      anomaly_detector: "anomaly-detector-predictor"
      predictive_analytics: "predictive-analytics-predictor"
      # ADD YOUR CUSTOM MODEL HERE:
      disk_failure_predictor: "disk-failure-predictor-predictor"
```

**3. Use from notebook:**

```python
result = client.detect_anomaly(
    model_name="disk-failure-predictor",
    instances=[[85.5, 5000, 365]]  # disk_usage, io_count, age_days
)
```


## 🎉 Congratulations!

You've completed the KServe Model Onboarding tutorial. You now know how to:

- ✅ Connect to the Coordination Engine
- ✅ Discover available models
- ✅ Make predictions via the KServe proxy
- ✅ Add custom models without code changes

### Quick Reference

```python
from utils.coordination_engine_client import get_client

client = get_client()
models = client.list_models()
result = client.detect_anomaly("model-name", [[features...]])
health = client.check_model_health("model-name")
```

### Next Steps

1. **Explore other notebooks:**
   - `02-anomaly-detection/` - Train your own models
   - `03-self-healing-logic/` - Integrate with remediation workflows
   - `04-model-serving/` - Deploy models to KServe

2. **Contribute!** Open issues or PRs at:
   - [Coordination Engine](https://github.com/tosin2013/openshift-coordination-engine)
   - [AIOps Platform](https://github.com/tosin2013/openshift-aiops-platform)
